In [ ]:
import matplotlib.pyplot as plt
import glob
import re
from Bio import PDB
from Bio.PDB.PDBIO import PDBIO
from Bio.PDB.StructureBuilder import StructureBuilder
import MDAnalysis as mda
import numpy as np

### First convert AF3 models from CIF to PDB

In [ ]:
# AF3 models are stored in AF3_models directory as cif files
AF3_models=glob.glob("AF3_models/*.cif")
# convert to PDB
for f in AF3_models:
    # get PDB file name
    fpdb=re.sub(".cif",".pdb",f)
    # convert to PDB
    parser = PDB.MMCIFParser()
    structure = parser.get_structure(structure_id="PDB", filename=f)
    io=PDBIO()
    io.set_structure(structure)
    io.save(fpdb)

### Load AF3 and AF2 models (PDB format)

In [ ]:
AF3_models=glob.glob("AF3_models/*.pdb")
# AF2 models are stored in AF2_models directory as pdb files
AF2_models=glob.glob("AF2_models/*.pdb")
# merge all
AF_models=AF2_models+AF3_models

### Extract per-residue pLDDT

In [ ]:
# prepare dictionary
pLDDT={}
for m in AF_models:
    pLDDT[m]=[]
    # read Bfactors where pLDDT is stored
    u = mda.Universe(m)
    # get Bfactors
    at_CA = u.select_atoms("name CA")
    pLDDT[m] = [ a.tempfactor for a in at_CA ]

### Plot per-residue pLDDT across all models

In [ ]:
# plot confidence ranges
plt.axhline(y = 90, color = 'b', linestyle = '--')
plt.axhline(y = 70, color = 'c', linestyle = '--') 
plt.axhline(y = 50, color = 'y', linestyle = '--') 
for key in pLDDT:
    l=re.sub(".pdb","",key)
    plt.plot(pLDDT[key], '-', label=l)
plt.xlabel('Residue ID')
plt.ylabel('pLDDT')
plt.legend(fontsize=6, frameon=False, ncols=2, loc="lower left")